### Домашнее задание: применяем t-sne

Модифицируйте файл *train.py* - добавьте в пайплайн обучения модели сжатие размерности до *n_components=2* с помощью t-sne и обучите модель **в докере** на "сжатых данных". Сохраните полученный объект `tsne_tansformer.pkl`, который умеет выполнять это задание.

Решением домашки считается модифицированный файл *train.py*

In [ ]:
import pickle
import os
import logging
from pathlib import Path

import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.manifold import TSNE

LOG_FORMAT = '%(asctime)s | %(levelname)-8s | %(filename)-25.25s:%(lineno)-4d | %(message)s'
log_filename = "/www/classifier/data/service.log"
logging.basicConfig(filename="log_filename", level=logging.INFO, format=LOG_FORMAT)

# загрузка данных
data_source = np.genfromtxt('data/client_segmentation.csv', delimiter=',', skip_header=1)
X = data_source[:, :3]
y = data_source[:, 3]
# обучение модели
tsne_transformer = TSNE(n_components=2)
X_tsne = tsne_transformer.fit_transform(X)
clf = DecisionTreeClassifier(max_depth=1, random_state=42)
clf.fit(X_tsne, y)

# сохраняем модель внутри контейнера в директории /www/classifier
with open('tsne_transformer.pkl', 'wb') as f:
    pickle.dump(tsne_transformer, f)
    logging.info('Модель для сжатия данных сохранена в %s' % Path().absolute())
with open('data/tsne_transformer.pkl', 'wb') as f:
    pickle.dump(tsne_transformer, f)

with open('clf.pkl', 'wb') as f:
    pickle.dump(clf, f)
    logging.info('Модель обучена и сохранена в %s' % Path().absolute())
with open('data/clf.pkl', 'wb') as f:
    pickle.dump(clf, f)
print(f"Модель обучена! Лог: {log_filename}")



### Домашнее задание: трансформация входных фичей на лету

Модифицируйте файл `service.py`: добавьте загрузку объекта для трансформации `tsne_tansformer.pkl` и применяйте её **в докере** для трансформации набора входных фич в сжатые:
<pre>
[x1, x2, x3] -> [x1_tsne, x2_tsne]
</pre>

Соответственно, predict надо выполнять на *сжатых* фичах


Решением домашки считается модифицированный файл *service.py*

In [ ]:
# --- ВАШ КОД ТУТ --
"""
Умеет выполнять классификацию клиентов по трём фичам

Запускаем из python3:
    python3 service.py
Проверяем работоспособность:
    curl http://127.0.0.1:5000/
"""
import json
import http.server
import logging
import os
import pickle
import socketserver
import sys
from http import HTTPStatus
from re import compile

import numpy as np
from sklearn.tree import DecisionTreeClassifier

# файл, куда посыпятся логи модели

LOG_FORMAT = '%(asctime)s | %(levelname)-8s | %(filename)-25.25s:%(lineno)-4d | %(message)s'
logging.basicConfig(filename="/www/classifier/data/service.log", level=logging.INFO, format=LOG_FORMAT)


def parse_params(params) -> dict:
    """
        Выдираем параметры из GET-запроса
    """
    params_list = params.split('&')
    params_dict = {}
    for param in params_list:
        key, value = param.split('=')
        params_dict[key] = float(value)
    return params_dict


class Handler(http.server.SimpleHTTPRequestHandler):
    """Простой http-сервер"""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_response(self) -> dict:
        """Пример запроса
        
        http://0.0.0.0:5000/classifier/?x1=1&x2=-2.2&x3=1.05&x4=7&x5=1.7&x6=17
        """
        response = {'ping': 'ok'}
        params_parsed = self.path.split('?')
        if len(params_parsed) == 2 and self.path.startswith('/classifier'):
            params = params_parsed[1]
            params_dict = parse_params(params)
            response = params_dict
            amount_of_elements = len(params_dict)
            if (amount_of_elements % 3 == 0) and (amount_of_elements > 3):
                user_features = np.zeros(amount_of_elements)
                i = 0
                for key in params_dict:
                    user_features[i] = params_dict[key]
                    i = i + 1
                user_features = user_features.reshape(int(amount_of_elements / 3), 3)
            else:
                user_features = np.zeros((2,amount_of_elements))
                i = 0
                for key in params_dict:
                    user_features[0,i] = params_dict[key]
                    user_features[1,i] = params_dict[key]
                    i = i + 1                   
            
            transform_user_features = data_compression_model.fit_transform(user_features)
            predicted_class = int(classifier_model.predict(transform_user_features)[0])
            logging.info('predicted_class %s' % predicted_class)
            response.update({'predicted_class': predicted_class})
        elif self.path.startswith('/ping/'):
            response = {'message': 'pong'}

        return response

    def do_GET(self):
        # заголовки ответа
        self.send_response(HTTPStatus.OK)
        self.send_header("Content-type", "application/json")
        self.end_headers()
        self.wfile.write(json.dumps(self.get_response()).encode())


logging.info('Загружаем обученные модели')
with open('/www/classifier/tsne_transformer.pkl', 'rb') as f_tsne:
    data_compression_model = pickle.load(f_tsne)
    logging.info('Модель для сжатия данных загружена: %s' % data_compression_model)

with open('/www/classifier/clf.pkl', 'rb') as f:
    classifier_model = pickle.load(f)
    logging.info('Модель загружена: %s' % classifier_model)

if __name__ == '__main__':
    classifier_service = socketserver.TCPServer(('', 5000), Handler)
    classifier_service.serve_forever()



# ------------------

### Домашнее задание: Используем Flask

Перепишите сервис на использование Flask. Вы можете взять готовый базовый образ с Flask, либо добавить установку в тот контейнер, который есть - это нужно сделать в Dockerfile


In [ ]:
# --- ВАШ КОД ТУТ --

FROM wrwrwr/flask-scipy

WORKDIR /www/classifier

RUN pip install -U scikit-learn

COPY docker-entrypoint.sh /usr/local/bin/docker-entrypoint.sh
RUN chmod +x /usr/local/bin/docker-entrypoint.sh

COPY train.py /www/classifier/train.py
COPY service.py /www/classifier/service.py

ENTRYPOINT ["docker-entrypoint.sh"]



# ------------------

## Домашнее задание: строим KNN

В реальной жизни KNN-рекомендатель не стоит делать на основе `sklearn.neighbors.NearestNeighbors` - есть готовые реализации, заточенные специально для построения рекомендательных систем. Хорошим примером такой реализации является [пакет implictit](). В рамках домашней работы предлагается разобраться с реализацией KNN-рекомендателя из этой библиотеки 

Почитайте документацию по модулю `implicit.nearest_neighbours.CosineRecommender`. Обучите KNN-рекомендатель и воспользуйтесь методом `recommend` для построения рекомендаций


In [1]:
# -- ВАШ КОД ТУТ --
import pandas as pd
import numpy as np

content_views = pd.read_csv(
    'recsys_data/content_views.zip', delimiter=',', header=0, compression='zip',
    names = ['user_id', 'content_id', 'view_duration', 'view_ts', 'dt', 'platform'],
    dtype = {'user_id': np.uint32, 'content_id': np.uint16, 'view_duration': np.uint16},
    parse_dates = [3, 4]
)



# -----------------

In [2]:
content_description = pd.read_csv(
    'recsys_data/content_description.zip', delimiter=',', header=0, compression='zip',
    names = ['content_id', 'origin_country', 'release_date', 'kinopoisk_rating', 'compilation_id', 'genre'],
    dtype = {'content_id': np.uint16},
    parse_dates = [2]
)

In [3]:
from sklearn.preprocessing import LabelEncoder

# кодируем индексы пользователей
user_encoder = LabelEncoder()
user_encoder.fit(content_views.user_id)

# ереиндексация контента
content_views = content_views.assign(
    user_index = user_encoder.transform(content_views.user_id)
)

# кодируем индексы контента
item_encoder = LabelEncoder()
item_encoder.fit(content_views.content_id)

# нова переиндексация
content_views = content_views.assign(
    item_index = item_encoder.transform(content_views.content_id)
)

In [4]:
from scipy.sparse import csr_matrix

num_users = content_views.user_index.max() + 1
num_items = content_views.item_index.max() + 1
num_interactions = content_views.shape[0]

user_item = csr_matrix(
    (np.ones(num_interactions),(content_views.item_index.values, content_views.user_index.values)),
    shape=(num_items, num_users)
)

In [5]:
import implicit
model = implicit.nearest_neighbours.CosineRecommender()
model.fit(user_item)

100%|█████████████████████████████████| 27012/27012 [00:01<00:00, 15962.16it/s]


In [6]:
random_user_index = np.random.choice(np.arange(start=0, stop=user_item.shape[1], step=1, dtype=np.uint32))
recommended = model.recommend(random_user_index, user_item, 15)

recommendations = []
scores = []

for item in recommended:
    idx, score = item
    recommendations.append(idx)
    scores.append(score)
    
print('user_index %d, recommendations: %s' % (random_user_index, recommendations))

user_index 893, recommendations: [238, 239, 236, 234, 233, 229, 242, 231, 240, 230, 241, 235, 232, 243, 225]


## Домашнее задание: Item to Item

Решите задачу c2c рекомендаций - вызовите метод `similar_items` для  *item_id=1*

In [7]:
# -- ВАШ КОД ТУТ --
item_id = 1
related = model.similar_items(item_id)
print("similar_items: ",related)

# ------------------

similar_items:  [(1, 0.9999999999999999), (0, 0.9952980993292889), (7, 0.9952863079166219), (2, 0.9951768998816896), (5, 0.9947149078229031), (20271, 0.9905321642238799), (20273, 0.9889862279059467), (8, 0.9831939420781712), (20269, 0.9822039197684757), (20272, 0.9792772453750449)]


### Домашнее задание: обучаем Implicit

Почитайте документацию по модулю implicit.als.AlternatingLeastSquares. Обучите ALS-рекомендатель и воспользуйтесь методом recommend для построения рекомендаций

In [8]:
model = implicit.als.AlternatingLeastSquares()
model.fit(user_item)
random_user_index = np.random.choice(np.arange(start=0, stop=user_item.shape[1], step=1, dtype=np.uint32))
recommended = model.recommend(random_user_index, user_item, 15)

recommendations = []
scores = []

for item in recommended:
    idx, score = item
    recommendations.append(idx)
    scores.append(score)
    
print('user_index %d, recommendations: %s' % (random_user_index, recommendations))


100%|████████████████████████████████████████| 15.0/15 [00:03<00:00,  4.31it/s]


user_index 357, recommendations: [15925, 16635, 16421, 15236, 18429, 14903, 15593, 22331, 14902, 24782, 14904, 14905, 14901, 14900, 13973]


### Домашнее задание на метрики

Даны два вектора - истинная история пользователя и объекты, которые считает релеватными ваша модель

Вычислите

* precision
* recall
* precision@5


In [9]:
import numpy as np

user_interactions = [47315, 30004, 36322,  8942, 30820,  6086,  9126,   332, 16289,
       39106, 39335, 48506, 48654,  9234, 29935,  2678, 36202, 22636, 18007, 39328, 15414, 30016, 35601,
    58409, 21313,   386, 16303, 4397, 19644, 51887, 21659, 36325, 53030,  7764, 50266, 58734, 53419, 24121,
    50806, 36092,  8868, 28037, 36131, 13561, 16298, 27508, 41722, 30189, 46490,  2676, 43328, 781, 48397,
    41369, 39324, 36381, 39635, 27710, 47837, 28525, 12024, 56604, 41664, 37387, 48507, 413, 33526, 20059,
    49781, 56648, 16283, 50805, 34254, 39325, 59374, 22620,  8865, 27512, 13875, 30011,  7621,
    10544, 28076, 29716, 30054, 20490, 29466, 16852, 39363, 34250, 7024, 33541,   263, 21267, 25690, 23020,
    41368, 53414,  2681, 30201] 

user_recs = [
    50820, 27781, 36131, 50812, 36092, 12024, 59155, 30042, 15414, 19882, 21659, 27849, 39328, 34240, 2681,
    21267, 50126, 58560, 7764, 49781
]

# --- ВАШ КОД ТУТ ---
k_total_num_views = len(user_interactions)
n_total_num_recs = len(user_recs)
m_viewed_recs = 0
for el in user_recs:
    if (el in user_interactions):
        m_viewed_recs += 1

recall = m_viewed_recs/k_total_num_views
precision = m_viewed_recs/n_total_num_recs

precision_5 = 0
i = 1
for el in user_interactions:
    if (el in user_recs):
        precision_5 += 1/i
    i += 1
precision_5 /= n_total_num_recs

print("Recall: ", recall)
print("Precision: ", precision)
print("Precision@5: ", precision_5)
# -------------------

Recall:  0.1
Precision:  0.5
Precision@5:  0.012958509750701858
